In [1]:
!pip install -U openai-agents duckduckgo-search

In [2]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI, OpenAIChatCompletionsModel, function_tool, trace
from duckduckgo_search import DDGS
#from datetime import datetime

In [3]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

In [10]:
# Configure the model
model = OpenAIChatCompletionsModel( 
    model="qwen2.5:7b",
    openai_client=AsyncOpenAI(base_url="http://localhost:11434/v1")
)

# check the connection, a simple agent for interacting with local LLM
agent = Agent(
    name="Chatbot",
    instructions="You are a local chatbot",
    model=model
)

result = Runner.run_sync(agent, "Tell me a joke.")

print(result.final_output)

Sure, here's a light one for you: Why don't scientists trust atoms?

Because they make up everything!


In [11]:
@function_tool
def get_news_articles(topic):
    print(f"Running DuckDuckGo news search for {topic}...")
    
    # DuckDuckGo search
    ddg_api = DDGS()
    results = ddg_api.text(f"{topic} ", max_results=5)
    if results:
        news_results = "\n\n".join([f"Title: {result['title']}\nURL: {result['href']}\nDescription: {result['body']}" for result in results])
        print(news_results)
        return news_results
    else:
        return f"Could not find news results for {topic}."

In [12]:
content_planner_agent = Agent(
    name="Content Planner",
    instructions="You are tasked with planning an engaging and informative blog post on {topic}. Your goal is to gather accurate, up-to-date information and structure the content",
    tools=[get_news_articles],
    model=model
)

In [13]:
writer_agent = Agent(
    name="Technical content writer",
    instructions="You are the technical content writer assigned to create a detailed and factually accurate blog post on: {topic}",
    model=model
)

In [14]:
# Agent workflow
def openai_agent_workflow(topic):
    print("Running Agent workflow...")

    # Step 1: Search and fetch articles by the content_planner_agent agent
    content_planner_response = Runner.run_sync(
        content_planner_agent,
        f"Get me the articles about {topic} "
    )
    
    # Access the content from Run Result object
    raw_articles = content_planner_response.final_output
    
    # Step 2: Pass articles to editor for final review
    edited_news_response = Runner.run_sync(
        writer_agent,
        raw_articles
    )
    
    # Access the content from RunResult object
    edited_article = edited_news_response.final_output
    
    print("Final article:")
    print(edited_article)
    
    return edited_article

In [15]:
topic = "Step by step example of installing Apache Ignite v3 on Docker."

print(openai_agent_workflow(topic))

Running Agent workflow...
Running DuckDuckGo news search for Step by step example of installing Apache Ignite v3 on Docker...
Title: Installing Using Docker | Ignite Documentation - Apache Ignite
URL: https://ignite.apache.org/docs/latest/installation/installing-using-docker
Description: By default, Ignite Docker image exposes the following ports: 11211, 47100, 47500, 49112. You can expose more ports as needed by adding -p  to the docker run command. For example, to connect a thin client to the node running inside a docker container, open port 10800:

Title: Apache Ignite: Setup Guide - DZone
URL: https://dzone.com/articles/apache-ignite-setup-guide
Description: Learn steps for binary installation, steps for building from sources, and steps for Maven to set up Apache Ignite. ... sources, Docker or cloud images, and Maven. There is also third-party support ...

Title: Getting started with Apache Ignite - installation and setup
URL: https://the-pi-guy.com/blog/getting_started_with_apache

In [18]:
# An editor agent for grammar and tone improvements.
editor_agent = Agent(
    name="Editor Agent",
    instructions="Proofread the given blog post for grammatical errors and alignment with human language for better readability while maintaining its original meaning.",
    handoffs=[content_planner_agent, writer_agent]
)

In [20]:
# Agent orchestration
async def main():
    result = await Runner.run(editor_agent, topic)
    print(result.final_output)